In [1]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np

# FOR WEBSCRAPING 
import requests
import requests_ftp
import requests_cache
import lxml
import re
import urllib
from bs4 import BeautifulSoup
from collections import Counter

# PLOTTING
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# CACHE
requests_cache.install_cache('urban_cache')

In [2]:
# WEBSCRAPING FAIRYGODBOSS
url_req = requests.get("https://fairygodboss.com/maternity-leave-resource-center")
html = url_req.text
soup = BeautifulSoup(html, 'lxml')

In [6]:
company_info_all = []
for i in range(80): # we know that "end" page on the site is 79
    base_url = 'https://fairygodboss.com/maternity-leave-resource-center'
    page = base_url + '?page=' + str(i)
    url_req = requests.get(page)
    html = url_req.text
    soup = BeautifulSoup(html, 'lxml')
    company_info = soup.select('a[href*="/company-overview/"]') #after inspecting element, this is where our info is 
    company_info_all.append(company_info)
#company_info_all[1]

In [11]:
# CREATE EMPTY LISTS FOR DATAFRAME: we need company name, industry, and 2 types of leave for each type of parent 
name = []
industry = []
mat_leave_pd = []
mat_leave_unp = []
pat_leave_pd = []
pat_leave_unp = []
for i in range(len(company_info_all)):
    company_info = company_info_all[i] #extract element from list
    for i in range(len(company_info)):
        s = company_info[i] #extract element from sublist to get relevant information
        #print s
        
        for i in range(len(s)):
            
            #s.find_all('li') has 6 different elements that we want, so we can use the indices 0:5 to find them
            name.append(s.find_all('li')[0].text.strip("\n"))
            industry.append(s.find_all('li')[1].text.split(' \n')[1].strip())
            mat_leave_pd.append(s.find_all('li')[2].text.split(' \n')[1].strip())
            mat_leave_unp.append(s.find_all('li')[3].text.split(' \n')[1].strip())
            pat_leave_pd.append(s.find_all('li')[4].text.split(' \n')[1].strip())
            pat_leave_unp.append(s.find_all('li')[5].text.split(' \n')[1].strip())

# CREATE DATAFRAME
df = pd.DataFrame({"name":name, "industry":industry, "maternity leave paid":mat_leave_pd, "maternity leave unpaid":mat_leave_unp, "paternity leave paid":pat_leave_pd, "paternity leave unpaid":pat_leave_unp})

# DROP DUPLICATES, KEEPING ONLY ONE OF EACH COMPANY
df.drop_duplicates(subset = ['name'], keep = 'first', inplace = True)

In [14]:
#df #we could poentially take the mean of each paid matleave and patleave and bargraph it according to each industry. hell yeah
df['industry'].value_counts()

Law Firm                                       88
Technology: Software                           81
Educational Services: College & Universi...    77
Technology: Consumer Internet                  77
Healthcare: Hospitals & Clinics                61
Technology: B2B Tech Services                  57
Finance: Diversified                           51
Retail: Shoes, Accessories and Apparel         44
Advertising                                    41
Pharmaceutical                                 38
Consulting Services                            36
Finance: Asset Management                      33
Technology: Manufacturing                      25
Hospitality: Restaurants                       24
Healthcare: Medical Devices                    23
Media: Diversified                             20
Consumer Packaged Goods: Packaged Foods        20
Telecommunications                             19
Natural Resources: Oil & Gas                   19
Insurance: Property & Casualty                 19


`df` is the dataframe I want to work with. The relevant information given by this dataset is industry, company name, and maternity and paternity leave (paid and unpaid for both). For the purposes of our analysis, we are going to focus on paid parental leave and see how this plays a role in social trends. 

First, as a summary, we want to determine how many types of industries there are in our dataframe and how many companies there are per industry. We can do this by just counting the unique values. A couple issues with this are that there are no duplicate industries at all, from a pandas perspective. When we look at it visually, we see that there are "groups" of industries such a "Technology: Security" and "Technology: Software", etc. First, we were going to create a column called `df['sector']` with industry names up till the ":" (using regex), but we decided it would be a better idea to just make the `sector` column by extracting the first word of every industry string. 

We want to look at only the top 10 industries in terms of company counts, to hopefully have a larger sample per industry (for example, industries such as "Arts & Entertainment" only have one company listed on fairygodboss, and in this case, although the samples are still small, these are all greater than 5, which is the best we're going to get with the information we have. 

From this subset, we have the most information about the nonprofit sector. We are interested in focusing in on the four following sectors: nonprofit technology, business, and government. 

In [5]:
df['industry']
df['numcompanies'] = [s.split(" ")[0] for s in df['industry']]
#df['sector'].value_counts()[:10]
sectordf = pd.DataFrame(df['numcompanies'].value_counts()[:10]) #dont have to do each sector; just compare nonprofit, tech, and business, and government 
sectordf
#differences column 

,numcompanies
Nonprofit:,22
Retail:,14
Healthcare:,11
Natural,11
Technology:,9
Industrial:,9
Business,7
Finance:,7
Educational,6
Government:,6


We can extract the information about the companies within these 10 industries, and determine the parental leave information. Since we're interested primarily in nonprofit, tech, business, and government, we'll look at those. 

In [106]:
#company %in% nonprofit 
df.head()
if df['sector'] in sector: 
    print true

KeyError: 'sector'